# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119736e+02     1.758680e+00
 * time: 0.11704587936401367
     1     1.078163e+01     9.304517e-01
 * time: 0.9971539974212646
     2    -1.136665e+01     1.044621e+00
 * time: 1.1298108100891113
     3    -3.381084e+01     8.066422e-01
 * time: 1.2921788692474365
     4    -4.742362e+01     5.508206e-01
 * time: 1.4419920444488525
     5    -5.689970e+01     2.017829e-01
 * time: 1.5918400287628174
     6    -5.980052e+01     1.227587e-01
 * time: 1.707468032836914
     7    -6.087691e+01     5.160136e-02
 * time: 1.8242650032043457
     8    -6.127824e+01     5.790651e-02
 * time: 1.9355900287628174
     9    -6.154535e+01     3.630522e-02
 * time: 2.054316997528076
    10    -6.175269e+01     2.845864e-02
 * time: 2.1717529296875
    11    -6.191712e+01     2.027259e-02
 * time: 2.2836239337921143
    12    -6.198916e+01     1.901359e-02
 * time: 2.4006710052490234
    13    -6.206704e+01     1.946930e-02
 * time: 2.52020692825317

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671076
    AtomicLocal         -18.8557718
    AtomicNonlocal      14.8522670
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485384 
    Xc                  -19.3336825

    total               -62.261666460970
